In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
%load_ext rpy2.ipython

In [3]:
%R library(ggplot2)

R[write to console]: Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang



array(['ggplot2', 'tools', 'stats', 'graphics', 'grDevices', 'utils',
       'datasets', 'methods', 'base'], dtype='<U9')

In [4]:
#function to make df readable into R
def sametype(df):
    cols = df.columns
    for index, val in enumerate(cols):
        if df[val].dtype == 'O':
            df[val] = df[val].astype('str')
    return df

attendance analysis
---

In [5]:
#read DATA in

#first read in attendance data
attendance_df = pd.read_csv('../data/historical/HomeGames.csv')

#read in franchises for active or not
franchises_df = pd.read_csv('../data/historical/TeamsFranchises.csv')

#park info
parks_df = pd.read_csv('../data/historical/Parks.csv')

#team info
teams_df = pd.read_csv('../data/historical/Teams.csv')

#postseason stats
post_season_df = pd.read_csv('../data/historical/SeriesPost.csv')

#stadium history
current_stadium_df = pd.read_csv('../data/stadiums/current_stadiums.csv')
past_stadium_df = pd.read_csv('../data/stadiums/past_stadiums.csv')

In [6]:
active_franchises_df = franchises_df.loc[franchises_df.active == 'Y']

In [8]:
#goes from 2895 rows to 2656 when filtering for only active
#also filter for last 50 years, from year 1969 and number of rows is 1398
active_teams_df = teams_df.loc[(teams_df.franchID.isin(active_franchises_df.franchID)) & (teams_df.yearID > 1968)]

In [9]:
active_teams_df = active_teams_df[['yearID', 'lgID', 'teamID', 'franchID', 'Rank', 'W', 'name', 'park', 'attendance']]

In [10]:
active_teams_df.head()

,yearID,lgID,teamID,franchID,Rank,W,name,park,attendance
1517,1969,NL,ATL,ATL,1,93,Atlanta Braves,Atlanta-Fulton County Stadium,1458320.0
1518,1969,AL,BAL,BAL,1,109,Baltimore Orioles,Memorial Stadium,1062069.0
1519,1969,AL,BOS,BOS,3,87,Boston Red Sox,Fenway Park II,1833246.0
1520,1969,AL,CAL,ANA,3,71,California Angels,Anaheim Stadium,758388.0
1521,1969,AL,CHA,CHW,5,68,Chicago White Sox,Comiskey Park,589546.0


In [11]:
post_season_df[['yearID', 'round', 'teamIDwinner', 'teamIDloser']].head()

,yearID,round,teamIDwinner,teamIDloser
0,1884,WS,PRO,NY4
1,1885,WS,CHN,SL4
2,1886,WS,SL4,CHN
3,1887,WS,DTN,SL4
4,1888,WS,NY1,SL4


In [26]:
#need to see if made playoffs and worldseries that year
with_series = pd.merge(active_teams_df, post_season_df[['yearID', 'round', 'teamIDwinner']], how = 'left', right_on = ['yearID', 'teamIDwinner'], left_on = ['yearID', 'teamID'])
with_series = pd.merge(with_series, post_season_df[['yearID', 'round', 'teamIDloser']], how = 'left', right_on = ['yearID', 'teamIDloser'], left_on = ['yearID', 'teamID'])
with_series.rename(columns={'round_x':'round won', 'round_y':'round lost'}, inplace=True)
with_series = with_series.drop(['teamIDwinner', 'teamIDloser'], axis=1)
#with_series

In [27]:
with_series = sametype(with_series)
with_series_over100 = with_series.loc[with_series.W > 100]
with_series_under100 = with_series.loc[with_series.W < 100]
with_series_under100.shape[0]

1406

add in column saying whether they made playoffs or not, due to join teams in the WS have 2 rows that season

In [28]:
with_series.loc[(with_series['round won']=='nan') & (with_series['round lost']=='nan'), 'playoff_boolean'] = 0

with_series.loc[(with_series.playoff_boolean.isna()), 'playoff_boolean'] = 1

In [32]:
with_series.head()

,yearID,lgID,teamID,franchID,Rank,W,name,park,attendance,round won,round lost,playoff_boolean
0,1969,NL,ATL,ATL,1,93,Atlanta Braves,Atlanta-Fulton County Stadium,1458320.0,nan,NLCS,1.0
1,1969,AL,BAL,BAL,1,109,Baltimore Orioles,Memorial Stadium,1062069.0,ALCS,WS,1.0
2,1969,AL,BOS,BOS,3,87,Boston Red Sox,Fenway Park II,1833246.0,nan,nan,0.0
3,1969,AL,CAL,ANA,3,71,California Angels,Anaheim Stadium,758388.0,nan,nan,0.0
4,1969,AL,CHA,CHW,5,68,Chicago White Sox,Comiskey Park,589546.0,nan,nan,0.0


get delta between wins and attendance in previous season

In [40]:
active_teams_df['prev_win'] = active_teams_df.sort_values(['franchID', 'yearID']).shift(1)['W']

In [43]:
active_teams_df['prev_attend'] = active_teams_df.sort_values(['franchID', 'yearID']).shift(1)['attendance']

In [45]:
#win diff from previous year calculated as how many more or less win this year than the previous year
#and attendance diff from previous year
active_teams_df['win_diff'] = active_teams_df['W'] - active_teams_df['prev_win']
active_teams_df['attend_diff'] = active_teams_df['attendance'] - active_teams_df['prev_attend']

In [48]:
#calcualte percentage change for attendance becuase stadium sizes vary, probably not necessary for wins since all out of 162
active_teams_df['attend_percent_change'] = active_teams_df['attendance']/active_teams_df['prev_attend'] -1

In [54]:
#change 1969 to N/A for win and attend diff and attend_percent_change
active_teams_df.loc[active_teams_df.yearID == 1969]['win_diff'] = np.nan
active_teams_df.loc[active_teams_df.yearID == 1969]['attend_diff'] = np.nan 
active_teams_df.loc[active_teams_df.yearID == 1969]['attend_percent_change'] = np.nan

/Users/rushilsheth/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/rushilsheth/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/rushilsheth/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [56]:
active_teams_df.loc[active_teams_df.yearID == 1969]

,yearID,lgID,teamID,franchID,Rank,W,name,park,attendance,prev_win,prev_attend,win_diff,attend_diff,attend_percent_change
1517,1969,NL,ATL,ATL,1,93,Atlanta Braves,Atlanta-Fulton County Stadium,1458320.0,82.0,2242695.0,11.0,-784375.0,-0.349747
1518,1969,AL,BAL,BAL,1,109,Baltimore Orioles,Memorial Stadium,1062069.0,90.0,2555781.0,19.0,-1493712.0,-0.584444
1519,1969,AL,BOS,BOS,3,87,Boston Red Sox,Fenway Park II,1833246.0,47.0,1564192.0,40.0,269054.0,0.172008
1520,1969,AL,CAL,ANA,3,71,California Angels,Anaheim Stadium,758388.0,NaN,NaN,NaN,NaN,NaN
1521,1969,AL,CHA,CHW,5,68,Chicago White Sox,Comiskey Park,589546.0,95.0,3181089.0,-27.0,-2591543.0,-0.814672
1522,1969,NL,CHN,CHC,2,92,Chicago Cubs,Wrigley Field,1674993.0,108.0,2895575.0,-16.0,-1220582.0,-0.421534
1523,1969,NL,CIN,CIN,3,89,Cincinnati Reds,Crosley Field,987991.0,62.0,1608817.0,27.0,-620826.0,-0.385890
1524,1969,AL,CLE,CLE,6,62,Cleveland Indians,Cleveland Stadium,619970.0,67.0,1629356.0,-5.0,-1009386.0,-0.619500
1525,1969,AL,DET,DET,2,90,Detroit Tigers,Tiger Stadium,1577481.0,91.0,3015880.0,-1.0,-1438399.0,-0.476942
1526,1969,NL,HOU,HOU,5,81,Houston Astros,Astrodome,1442995.0,63.0,811104.0,18.0,631891.0,0.779051


stadium stuff
---